# <a id="root">Business questions?</a>
>Use [`e-commerce dataset`](https://www.kaggle.com/carrie1/ecommerce-data) to answer the following business questions:
<br><br>1.	Using Data Frame, find out the [top 10 countries where people have spent more money on retail store](#1) on whole data set from starting date to ending date. Output to Hadoop as csv to location `/output/top-10-countries` directory
<br><br>2.	Using Data Frame, find out [Total Quantities of Items sold per StockCode per month](#2), write the output to the HDFS table.
<br><br>3.	Using Spark SQL, what's the [monthly turnover of the store?](#3)
<br><br>4.	Using Spark SQL, find out [monthly turnover of the each stock sold.](#4)

<br>

## SparkSession Setup

>__Run Spark Cluster Master/Workers__
```bash
spark-class org.apache.spark.deploy.master.Master  # 1
spark-class org.apache.spark.deploy.worker.Worker spark://192.168.11.77:7077  # 2
```
<br>__Register Cluster Master with Spark Driver__
```bash
pyspark --master spark://192.168.11.77:7077 --executor-memory 2G --executor-cores 2 --num-executors 2
```

In [1]:
import findspark
findspark.init()

In [2]:
"""
SparkSession conf
"""

from pyspark.conf import SparkConf

config = SparkConf()
conf = \
(
    config
    .setMaster("spark://192.168.11.77:7077").setAppName("E-COMMERCE:CLUSTER")
    .set("spark.executor.memory", "2g")
    .set("spark.executor.cores", 4)
    .set("spark.cores.max", 4)
    .set("spark.driver.memory", "2g")
)

In [3]:
"""
SparkSession
"""

from pyspark.sql import SparkSession

ss = (
    SparkSession
    .builder
    .config(conf=conf)
    .getOrCreate()
)

<br><br>

## Read e-commerce data

In [4]:
"""
Read CSV from HDFS
"""

import datetime as dt
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, IntegerType, DoubleType, StringType, DateType
from pyspark.sql.functions import col, asc, desc, count, sum, avg, to_date


schema_ecomm = (
    StructType()
    .add("InvoiceNo", StringType(), True)
    .add("StockCode", StringType(), True)
    .add("Description", StringType(), True)
    .add("Quantity", IntegerType(), True)
    .add("InvoiceDate", DateType(), True)
    .add("UnitPrice", DoubleType(), True)
    .add("CustomerId", StringType(), True)
    .add("Country", StringType(), True)
)

df_ecomm_full = (
    ss.read
    .format("csv")
    .option("header", True)
    .option("dateFormat", "MM/dd/yyyy HH:mm")
    .schema(schema_ecomm)
    .load("hdfs://192.168.93.128:9000/input/e-commerce/data.csv")
)


"""
DROP un-necessary data/columns by choosing only what's necessary
"""
df_ecomm = df_ecomm_full[["InvoiceNo", "StockCode", "Country", "CustomerId", "Quantity", "UnitPrice", "InvoiceDate"]]

In [5]:
df_ecomm_full.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Country: string (nullable = true)



In [6]:
df_ecomm_full.show(2)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-12-01|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-12-01|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
only showing top 2 rows



In [7]:
df_ecomm_full = (
    df_ecomm_full
    .withColumn("Invoice_Date", F.to_date(col("InvoiceDate"), "dd/MM/yyyy"))
    .withColumn("Year", F.year("Invoice_Date"))
    .withColumn("Month", F.month("Invoice_Date"))
    .drop("InvoiceDate", "Description")
    .withColumn("Sales", col("Quantity") * col("UnitPrice"))
)
df_ecomm_full.show(1)

+---------+---------+--------+---------+----------+--------------+------------+----+-----+------------------+
|InvoiceNo|StockCode|Quantity|UnitPrice|CustomerId|       Country|Invoice_Date|Year|Month|             Sales|
+---------+---------+--------+---------+----------+--------------+------------+----+-----+------------------+
|   536365|   85123A|       6|     2.55|     17850|United Kingdom|  2010-12-01|2010|   12|15.299999999999999|
+---------+---------+--------+---------+----------+--------------+------------+----+-----+------------------+
only showing top 1 row



<br><br>

## <a id="1">[Top 10 countries by date/sales](#root)</a>

In [8]:
show_n = 10

In [9]:
from pyspark.sql.window import Window

df_ecomm = df_ecomm_full[["Country", "Invoice_Date", "Sales"]]

df_top10_sales_date = (
    df_ecomm
    .groupby([df_ecomm.Invoice_Date, df_ecomm.Country])
    .sum()
    .withColumnRenamed("sum(Sales)", "TotalSales")
)


window = (
    Window.partitionBy(df_top10_sales_date.Invoice_Date)
    .orderBy(df_top10_sales_date.TotalSales.desc())
)


df_top10_sales_date = (
    df_top10_sales_date
    .select("*", F.rank().over(window).alias("rank"))
    .filter(col("rank") <= 10)
    .sort(F.asc("Invoice_Date"), F.desc("TotalSales"))
)

df_top10_sales_date.show(show_n)

+------------+--------------+------------------+----+
|Invoice_Date|       Country|        TotalSales|rank|
+------------+--------------+------------------+----+
|  2010-12-01|United Kingdom| 54615.15000000007|   1|
|  2010-12-01|        Norway|1919.1400000000008|   2|
|  2010-12-01|        France|            855.86|   3|
|  2010-12-01|          EIRE| 555.3799999999999|   4|
|  2010-12-01|     Australia|            358.25|   5|
|  2010-12-01|   Netherlands|192.60000000000002|   6|
|  2010-12-01|       Germany|139.18000000000004|   7|
|  2010-12-02|United Kingdom| 46053.92999999993|   1|
|  2010-12-02|       Germany|138.35000000000002|   2|
|  2010-12-02|          EIRE|              15.0|   3|
+------------+--------------+------------------+----+
only showing top 10 rows



In [10]:
"""
Write to HDFS as a CSV
Output directory: /output/e-commerce/top-10-countries
"""

(
    df_top10_sales_date
    .coalesce(1)
    .write.mode('overwrite')
    .option("header", True)
    .csv("hdfs://192.168.93.128:9000/output/e-commerce/top-10-countries")
)

```bash
ubuntu@ubuntuvm:~$ hdfs dfs -ls /output/e-commerce/top-10-countries
Found 2 items
-rw-r--r--   3 Administrator supergroup          0 2021-05-22 16:08 /output/e-commerce/top-10-countries/_SUCCESS
-rw-r--r--   3 Administrator supergroup      63467 2021-05-22 16:08 /output/e-commerce/top-10-countries/part-00000-3fada342-b26d-42e4-98d1-dab202666412-c000.csv

ubuntu@ubuntuvm:~$
```

### _Verification_

In [11]:
ss.read.format("csv").option("header", True).load("hdfs://192.168.93.128:9000/output/e-commerce/top-10-countries/").show()

+------------+--------------+------------------+----+
|Invoice_Date|       Country|        TotalSales|rank|
+------------+--------------+------------------+----+
|  2010-12-01|United Kingdom| 54615.15000000007|   1|
|  2010-12-01|        Norway|1919.1400000000008|   2|
|  2010-12-01|        France|            855.86|   3|
|  2010-12-01|          EIRE| 555.3799999999999|   4|
|  2010-12-01|     Australia|            358.25|   5|
|  2010-12-01|   Netherlands|192.60000000000002|   6|
|  2010-12-01|       Germany|139.18000000000004|   7|
|  2010-12-02|United Kingdom| 46053.92999999993|   1|
|  2010-12-02|       Germany|138.35000000000002|   2|
|  2010-12-02|          EIRE|              15.0|   3|
|  2010-12-03|United Kingdom|39985.439999999864|   1|
|  2010-12-03|          EIRE|2576.8500000000004|   2|
|  2010-12-03|         Spain|             620.0|   3|
|  2010-12-03|       Germany| 555.5300000000001|   4|
|  2010-12-03|         Italy|427.79999999999995|   5|
|  2010-12-03|        France

<br><br>

## <a id="2">[Quantities sold by Month/StockCode](#root)</a>

In [12]:
df_month_stockcode = df_ecomm_full[["Month", "StockCode", "Quantity"]]
df_month_stockcode.show(2)

+-----+---------+--------+
|Month|StockCode|Quantity|
+-----+---------+--------+
|   12|   85123A|       6|
|   12|    71053|       6|
+-----+---------+--------+
only showing top 2 rows



In [13]:
"""
Although the requirement isn't to sort by the most sales by StockCode,
I still prefer to see who were at the top, month-wise.
Sorted, it'll be easier to spot highest/least selling StockCode.
"""

df_month_stockcode = (
    df_month_stockcode
    .groupby("Month", "StockCode")
    .agg(F.sum("Quantity").alias("TotalQuantity"))
    .sort(F.asc("Month"), F.desc("TotalQuantity"))
)
df_month_stockcode.show(show_n)

+-----+---------+-------------+
|Month|StockCode|TotalQuantity|
+-----+---------+-------------+
|    1|    37413|         5568|
|    1|   85123A|         5522|
|    1|    21212|         3409|
|    1|    21108|         3246|
|    1|    17003|         3181|
|    1|    22469|         2919|
|    1|   85099B|         2639|
|    1|   79063D|         2560|
|    1|    21175|         2410|
|    1|    40016|         2128|
+-----+---------+-------------+
only showing top 10 rows



In [14]:
"""
Write to HDFS as a table, with month-wise directory
Output directory: /output/e-commerce/month_stockcode_quantity/Month=XX
"""

(
    df_month_stockcode
    .coalesce(1)
    .write.mode('overwrite')
    .option("header", True)
    .partitionBy("Month")  # --> Remove to write as a single file
    .saveAsTable(
        "month_stockcode_quantity",
        path="hdfs://192.168.93.128:9000/output/e-commerce/month_stockcode_quantity"
    )
)

```bash
ubuntu@ubuntuvm:~$ hdfs dfs -ls /output/e-commerce/month_stockcode_quantity
Found 13 items
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=1
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=10
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=11
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=12
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=2
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=3
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=4
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=5
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=6
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=7
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=8
drwxr-xr-x   - Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=9
-rw-r--r--   3 Administrator supergroup          0 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/_SUCCESS

ubuntu@ubuntuvm:~$ hdfs dfs -ls /output/e-commerce/month_stockcode_quantity/Month=1
Found 1 items
-rw-r--r--   3 Administrator supergroup      17789 2021-05-22 15:48 /output/e-commerce/month_stockcode_quantity/Month=1/part-00000-4f9b094e-8ea0-49dc-b878-df1010d81bf4.c000.snappy.parquet

ubuntu@ubuntuvm:~$
```

### _Verification_

In [15]:
ss.read.load("hdfs://192.168.93.128:9000/output/e-commerce/month_stockcode_quantity").show()

+---------+-------------+-----+
|StockCode|TotalQuantity|Month|
+---------+-------------+-----+
|    22197|         8929|   12|
|    84077|         6562|   12|
|    22086|         4542|   12|
|    21212|         4535|   12|
|    23084|         4298|   12|
|   85123A|         4039|   12|
|    84879|         3701|   12|
|    22834|         3481|   12|
|    21137|         3418|   12|
|    20668|         3381|   12|
|    21915|         3348|   12|
|    22616|         3317|   12|
|   85099B|         3216|   12|
|    22355|         3109|   12|
|    22492|         3002|   12|
|    22693|         2874|   12|
|    84946|         2729|   12|
|    22189|         2684|   12|
|    22961|         2677|   12|
|    22188|         2510|   12|
+---------+-------------+-----+
only showing top 20 rows



<br><br>


##  <a id="3">[Monthly Turnover of Store](#root)</a>

In [16]:
df_ecomm_full.createOrReplaceTempView("ecomm")

In [17]:
"""
NOTE: e-commerce dataset utilized has data for only 1 year
"""

ss.sql("SELECT DISTINCT Year, Month FROM ecomm ORDER BY 1, 2").show()

+----+-----+
|Year|Month|
+----+-----+
|2010|   12|
|2011|    1|
|2011|    2|
|2011|    3|
|2011|    4|
|2011|    5|
|2011|    6|
|2011|    7|
|2011|    8|
|2011|    9|
|2011|   10|
|2011|   11|
|2011|   12|
+----+-----+



In [18]:
"""
Turnover ratio = CostOfGoodsSold / AverageOfInventory for the month
Output: Displayed in-line
"""

ss.sql("""
SELECT Year,
       Month,
       SUM(Sales) AS Sales,
       SUM(Quantity) AS InventoryEnd,
       LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year ORDER BY Month) AS InventoryBeginning,
       ((SUM(Quantity) + LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year ORDER BY Month))/2) AS InventoryAverage,
       SUM(Sales) / ((SUM(Quantity) + LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year ORDER BY Month))/2) AS Turnover
FROM ecomm
GROUP BY 1, 2
ORDER BY 1, 2
""").show()

+----+-----+------------------+------------+------------------+----------------+------------------+
|Year|Month|             Sales|InventoryEnd|InventoryBeginning|InventoryAverage|          Turnover|
+----+-----+------------------+------------+------------------+----------------+------------------+
|2010|   12| 748957.0199999774|      342228|                 0|        171114.0| 4.376947648935666|
|2011|    1| 560000.2600000234|      308966|                 0|        154483.0|3.6249960189795862|
|2011|    2| 498062.6500000268|      277989|            308966|        293477.5|1.6971067628694765|
|2011|    3| 683267.0800000189|      351872|            277989|        314930.5|2.1695805264971764|
|2011|    4| 493207.1210000137|      289098|            351872|        320485.0|1.5389397974944654|
|2011|    5|   723333.51000001|      380391|            289098|        334744.5| 2.160852560684373|
|2011|    6| 691123.1200000228|      341623|            380391|        361007.0|1.9144313545167346|


<br><br>


##  <a id="4">[Monthly Turnover by StockCode](#root)</a>

In [19]:
"""
How many StockCodes does the store have?
"""

ss.sql("SELECT DISTINCT StockCode FROM ecomm").count()

4070

In [20]:
ss.sql("""
SELECT Year,
       StockCode,
       Month,
       COUNT(*) AS month_stock_total,
       LAG(COUNT(*), 1, 0) OVER (PARTITION BY Year, StockCode ORDER BY Month) AS month_stock_total_previous
  FROM ecomm
 GROUP BY 1, 2, 3
 ORDER BY 1, 2, 3
""")

DataFrame[Year: int, StockCode: string, Month: int, month_stock_total: bigint, month_stock_total_previous: bigint]

In [21]:
turnover_stockcode_monthly = \
ss.sql("""
SELECT Year,
       StockCode,
       Month,
       SUM(Sales) AS Sales,
       SUM(Quantity) AS InventoryEnd,
       LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year, StockCode ORDER BY Month) AS InventoryBeginning,
       ((SUM(Quantity) + LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year, StockCode ORDER BY Month))/2) AS InventoryAverage,
       SUM(Sales) / ((SUM(Quantity) + LAG(SUM(Quantity), 1, 0) OVER (PARTITION BY Year, StockCode ORDER BY Month))/2) AS Turnover
FROM ecomm
GROUP BY 1, 2, 3
ORDER BY 1, 2, 3
""")

turnover_stockcode_monthly.count()

34691

In [22]:
"""
Writing to HDFS & reading from there is easier due to the number of records constraint.
Output directory: /output/e-commerce/turnover_stockcode_monthly
"""

(
    turnover_stockcode_monthly
    .coalesce(1)
    .write.mode('overwrite')
    .option("header", True)
    .saveAsTable(
        "turnover_stockcode_monthly",
        path="hdfs://192.168.93.128:9000/output/e-commerce/turnover_stockcode_monthly"
    )
)

```bash
ubuntu@ubuntuvm:~$ hdfs dfs -ls /output/e-commerce/
Found 1 items
drwxr-xr-x   - Administrator supergroup          0 2021-05-23 13:00 /output/e-commerce/turnover_stockcode_monthly

ubuntu@ubuntuvm:~$ hdfs dfs -ls /output/e-commerce/turnover_stockcode_monthly
Found 2 items
-rw-r--r--   3 Administrator supergroup          0 2021-05-23 13:00 /output/e-commerce/turnover_stockcode_monthly/_SUCCESS
-rw-r--r--   3 Administrator supergroup     184279 2021-05-23 13:00 /output/e-commerce/turnover_stockcode_monthly/part-00000-46336529-17c9-4d5e-9e91-fca03582f349-c000.snappy.parquet

ubuntu@ubuntuvm:~$
```

### _Verification_

In [23]:
ss.read.load("hdfs://192.168.93.128:9000/output/e-commerce/turnover_stockcode_monthly").show()

+----+---------+-----+------------------+------------+------------------+----------------+------------------+
|Year|StockCode|Month|             Sales|InventoryEnd|InventoryBeginning|InventoryAverage|          Turnover|
+----+---------+-----+------------------+------------+------------------+----------------+------------------+
|2010|    10002|   12| 234.4099999999999|         251|                 0|           125.5|1.8678087649402384|
|2010|    10120|   12|              3.36|          16|                 0|             8.0|              0.42|
|2010|   10123C|   12|              0.65|           1|                 0|             0.5|               1.3|
|2010|   10124A|   12|              1.68|           4|                 0|             2.0|              0.84|
|2010|   10124G|   12|               2.1|           5|                 0|             2.5|0.8400000000000001|
|2010|    10125|   12| 90.85999999999999|         154|                 0|            77.0|1.1799999999999997|
|2010|    

<br><br>

### How may partitions in use?

In [24]:
turnover_stockcode_monthly.rdd.getNumPartitions()

200

In [25]:
ss.stop()